In [2]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [3]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
df_w_admission_rate = df.copy(deep=True)

In [4]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": 0, "Wait-Listed": 0, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [5]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)

for col in ['cost_attendance', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)

df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)

In [6]:
print(len(df.index))

44042


In [7]:
whole_df = df

def split_df_on(feature=None):
    split_dfs = {}
    if feature:
        for val in whole_df[feature].unique():
            split_dfs[val] = (whole_df.loc[whole_df[feature]==val])
    else:
        split_dfs = {'all': whole_df}
    return split_dfs
df.sample(n=100)

,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,gpa_uw,class_rank,status,eaed,legacy,athlete,test_score,avg_test_score,in_state_tuition,out_state_tuition
43018,"University of California, Irvine",1,3.97,23886,37,1290,16.,0.0,52.7,2014,...,3.80,0.100000,2,0,0,0,0.722222,0.805556,33826,61840
74925,Bridgewater College,9,3.44,32931,52,120,1.3,1.0,54.4,2020,...,3.88,1.000000,2,0,0,0,0.750000,0.611111,51310,51310
24481,Blackburn College,8,3.26,11673,52,32,1.2,1.0,54.0,2019,...,3.80,0.097087,2,0,0,0,0.611111,0.583333,31302,31302
26958,University of Michigan,0,3.84,27050,29,2791,7.1,0.0,49.8,2009,...,4.00,0.015000,2,0,0,0,0.972222,0.805556,29526,62176
16238,Marymount Manhattan College,5,3.44,19019,79,91,4.1,1.0,77.3,2014,...,4.00,0.050000,2,0,0,0,0.694444,0.694444,58110,58110
68500,University of Nebraska - Lincoln,41,3.56,14958,64,1117,8.9,0.0,47.5,2018,...,3.90,0.006452,2,0,0,0,0.861111,0.694444,24935,40235
86649,University of North Carolina at Wilmington,24,4.18,8348,67,647,1.5,0.0,62.2,2020,...,3.93,0.055556,2,1,0,0,0.666667,0.694444,23907,37923
33938,Harvard College,47,4.18,55354,5,975,11.,1.0,47.7,2017,...,4.00,0.002000,0,0,0,0,0.972222,0.944444,69600,69600
85371,North Carolina State University,24,3.72,13222,51,2245,4.7,0.0,45.4,2014,...,3.33,0.304348,2,0,0,0,0.611111,0.805556,23976,43320
83093,Virginia Polytechnic Institute and State Unive...,9,3.97,17549,70,1806,6.4,0.0,43.1,2020,...,3.85,0.134615,2,0,0,0,0.666667,0.750000,21920,39099


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

split_on = None
for val, df_s in split_df_on(split_on).items():
    k = 1
    
    df0 = df_s[df.status == 0]
    df2 = df_s[df.status == 2].sample(n = int(k*len(df0)))
    
    df_final = pd.concat([df0, df2])
    schools = df_final['school']
    df_final = df_final.drop('school', axis=1)
    X = df_final.drop('status', axis=1)
    y = df_final['status']
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(21, 21, 21),max_iter=1000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))

    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_train, y_train)
    score = logisticRegr.score(X_test, y_test)
    print(f"Logistic Regression Score for {split_on} {val}: " + str(score))



[[1841  375]
 [ 480 1699]]
             precision    recall  f1-score   support

          0       0.79      0.83      0.81      2216
          2       0.82      0.78      0.80      2179

avg / total       0.81      0.81      0.81      4395

Logistic Regression Score for None all: 0.7954493742889648


In [9]:
old_x = X.copy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

from collections import defaultdict
experimental_acceptance_rates = defaultdict(lambda: defaultdict(int))
for res, school in zip(mlp.predict(X), schools):
    if res == 0:
        experimental_acceptance_rates[school]['reject'] += 1
    elif res == 0:
        experimental_acceptance_rates[school]['accept'] += 1


In [10]:
experimental_acceptance_rates

defaultdict(<function __main__.<lambda>>,
            {'Abilene Christian University': defaultdict(int, {'reject': 4}),
             'Adams State University': defaultdict(int, {'reject': 2}),
             'Adelphi University': defaultdict(int, {'reject': 17}),
             'Adrian College': defaultdict(int, {'accept': 1, 'reject': 1}),
             'Agnes Scott College': defaultdict(int,
                         {'accept': 1, 'reject': 7}),
             'Alabama State University': defaultdict(int,
                         {'accept': 2, 'reject': 6}),
             'Albany College of Pharmacy and Health Sciences': defaultdict(int,
                         {'reject': 4}),
             'Albany State University': defaultdict(int,
                         {'accept': 1, 'reject': 3}),
             'Albertus Magnus College': defaultdict(int, {'reject': 1}),
             'Albion College': defaultdict(int, {'accept': 1}),
             'Albright College': defaultdict(int, {'accept': 2, 'reject': 

In [11]:
compare_vs_ground_truth = defaultdict(str)

for (index, row), school in zip(old_x.iterrows(), schools):
    accept_rate = None
    if experimental_acceptance_rates[school]['accept'] != 0 and experimental_acceptance_rates[school]['accept'] != 0:
        accept_rate = experimental_acceptance_rates[school]['accept']/(experimental_acceptance_rates[school]['accept']+experimental_acceptance_rates[school]['reject'])
    elif experimental_acceptance_rates[school]['accept'] == 0:
        accept_rate = 0
    elif experimental_acceptance_rates[school]['reject'] == 0:
        accept_rate = 1
    
    compare_vs_ground_truth[school] = [accept_rate, int(row['admission_rate'])/100]

In [12]:
compare_vs_ground_truth

defaultdict(str,
            {'Abilene Christian University': [0, 0.51],
             'Adams State University': [0, 0.99],
             'Adelphi University': [0, 0.73],
             'Adrian College': [0.5, 0.56],
             'Agnes Scott College': [0.125, 0.66],
             'Alabama State University': [0.25, 0.98],
             'Albany College of Pharmacy and Health Sciences': [0, 0.67],
             'Albany State University': [0.25, 0.5],
             'Albertus Magnus College': [0, 0.67],
             'Albion College': [1.0, 0.71],
             'Albright College': [0.2222222222222222, 0.5],
             'Alcorn State University': [0, 0.79],
             'Alfred University': [0.25, 0.63],
             'Allegheny College': [0.125, 0.68],
             'Alma College': [0, 0.64],
             'Alvernia University': [0, 0.74],
             'American International College': [0, 0.72],
             'American University': [0.5775862068965517, 0.29],
             'Anderson University': [0, 0.